In [ ]:
#get all the submissions from the day 
#get 10 with most twitter citations
#summarize and post them 

In [2]:
"""
based on ET - every day one hour past 00:00 past ET i will tweet it 

go to certain website 

1. get date of day before 
2. open first page and check if date is equals to target date if yes add all the paper urls from the page to the list and if date is older then target date exit
3. open all pages, get the number of tweet impressions, get the url to the pdf, title and subject area
4. make summary and post
"""
from datetime import datetime, timedelta
import pytz
import requests
from bs4 import BeautifulSoup
import time

# Get date from yesterday ET time
eastern = pytz.timezone('US/Eastern')
yesterday = datetime.now(eastern) - timedelta(days=1)
yesterday_date = yesterday.strftime('%Y-%m-%d')
print("Yesterday's date in ET:", yesterday_date)

page_data = []

def iterate_biorxiv_pages():
    page_number = 0
    while True:
        url = f"https://www.biorxiv.org/content/early/recent?page={page_number}"
        print(url)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Get the date based on the X path
        date_element = soup.select_one('#block-system-main > div > div > div > div:nth-child(2) > div:nth-child(1) > div > div > div > div > div:nth-child(1) > h3')
        date_text = date_element.get_text(strip=True)
        
        # Reformat the date to match the format of our yesterday_date
        page_date = datetime.strptime(date_text, '%B %d, %Y').strftime('%Y-%m-%d')

        print("page date: ", page_date)
        
        # Check if the date is older than the target date
        if page_date < yesterday_date:
            break
        
        # Skip the page if the date is newer than the target date
        if page_date > yesterday_date:
            page_number += 1
            continue

        # Extract all the URLs from the page
        article_blocks = soup.find_all('div', class_='highwire-cite highwire-cite-highwire-article highwire-citation-biorxiv-article-pap-list-overline clearfix')
        
        for block in article_blocks:
            title_element = block.find('span', class_='highwire-cite-title')
            if title_element:
                link_element = title_element.find('a', class_='highwire-cite-linked-title')
                if link_element and 'href' in link_element.attrs:
                    paper_url = "https://www.biorxiv.org" + link_element['href']
                    page_data.append(paper_url)
        
        #here i can extract all the data

        # If date is target date, get all the data and save it in list
        print(page_date)
        # Add logic to extract and save data here
        
        page_number += 1
    return page_data

page_data = iterate_biorxiv_pages()

Yesterday's date in ET: 2024-08-30
https://www.biorxiv.org/content/early/recent?page=0
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=1
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=2
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=3
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=4
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=5
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=6
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=7
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=8
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=9
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=10
page date:  2024-08-30
2024-08-30
https://www.biorxi

In [13]:
import asyncio
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import nest_asyncio

# Apply the nest_asyncio patch
nest_asyncio.apply()

# Initialize an empty list to store the dictionaries
tweet_data_list = []

# Function to generate the target date
def get_yesterday_date():
    yesterday = datetime.now() - timedelta(days=1)
    return yesterday.strftime('%Y-%m-%d')

yesterday_date = get_yesterday_date()

# Asynchronous function to fetch and parse a single URL
async def fetch_and_parse(url, context):
    print(f"Fetching URL: {url}")
    retries = 3
    for attempt in range(retries):
        try:
            page = await context.new_page()
            await page.goto(url, timeout=60000)  # Wait up to 60 seconds for the page to load
            await page.wait_for_selector('#count_twitter', timeout=60000)  # Wait up to 60 seconds for the element
            content = await page.content()
            await page.close()
            break  # Exit the retry loop if successful
        except PlaywrightTimeoutError as e:
            print(f"Timeout error on {url}: {e}")
        except Exception as e:
            print(f"Error on {url}: {e}")
        if attempt < retries - 1:
            print(f"Retrying {url} (attempt {attempt + 1}/{retries})")
        else:
            print(f"Failed to fetch {url} after {retries} attempts")
            return

    soup = BeautifulSoup(content, 'html.parser')
    
    # Extract the date from the page
    date_element = soup.select_one('#block-system-main > div > div > div > div > div:nth-child(2) > div > div > div:nth-child(3) > div')
    if date_element:
        date_text = date_element.get_text(strip=True)
        # Remove the 'Posted\xa0' prefix if it exists
        if date_text.startswith('Posted\xa0'):
            date_text = date_text.replace('Posted\xa0', '')
        # Remove the period at the end of the date string
        if date_text.endswith('.'):
            date_text = date_text[:-1]
        try:
            page_date = datetime.strptime(date_text, '%B %d, %Y').strftime('%Y-%m-%d')
            print(f"Parsed page date: {page_date}")  # Debug print
        except ValueError as e:
            print(f"Error parsing date on {url}: {e}")
            return
        
        # Check if the date matches the target date
        if page_date == yesterday_date:
            print(f"Date matches target date: {page_date}")  # Debug print
            # Extract the number of tweets using a CSS selector
            tweet_element = soup.select_one('#count_twitter')
            print("tweet_element: ", tweet_element)  # Debug print
            if tweet_element:
                tweet_count = tweet_element.get_text(strip=True)
                print(f"Extracted tweet count: {tweet_count}")  # Debug print
                
                # Create a dictionary with the URL as key and tweet count as value
                tweet_data = {url: tweet_count}
                
                # Add the dictionary to the list
                tweet_data_list.append(tweet_data)
            else:
                print(f"Tweet element not found on {url}")
        else:
            print(f"Date does not match target date on {url}")
    else:
        print(f"Date element not found on {url}")

# Main asynchronous function to handle multiple requests in batches
async def main(urls, batch_size=50, delay=5):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        
        for i in range(0, len(urls), batch_size):
            batch = urls[i:i + batch_size]
            print(f"Processing batch: {batch}")
            context = await browser.new_context()
            tasks = [fetch_and_parse(url, context) for url in batch]
            await asyncio.gather(*tasks)
            await context.close()
            await asyncio.sleep(delay)  # Add a delay between batches
        
        await browser.close()

# List of URLs to process
urls = page_data

# Run the main function
await main(urls)

# Print the list of dictionaries
print(tweet_data_list)

Processing batch: ['https://www.biorxiv.org/content/10.1101/2024.08.30.610472v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.610471v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.609158v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.609923v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.610437v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.610386v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.610159v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.610496v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.610464v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.609148v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.610507v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.610432v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.610421v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.610463v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.610303v1', 'https://www.biorxiv.org/content/10.

In [12]:
# Function to find the top ten URLs with the most tweets
def get_top_ten_tweets(tweet_data_list):
    top_ten_tweets = []
    
    # Flatten the list of dictionaries into a single list of tuples (url, tweet_count)
    flattened_tweets = [(url, int(tweet_count)) for data in tweet_data_list for url, tweet_count in data.items()]
    
    # Sort the list in descending order based on tweet count
    sorted_tweets = sorted(flattened_tweets, key=lambda x: x[1], reverse=True)
    
    # Select the top ten URLs with the most tweets
    top_ten_tweets = sorted_tweets[:10]
    
    return top_ten_tweets

# Get the top ten URLs with the most tweets
top_ten_tweets = get_top_ten_tweets(tweet_data_list)
print("Top ten URLs with the most tweets:")
for url, tweet_count in top_ten_tweets:
    print(f"URL: {url}, Tweets: {tweet_count}")

Top ten URLs with the most tweets:
URL: https://www.biorxiv.org/content/10.1101/2023.11.27.568910v2, Tweets: 103
URL: https://www.biorxiv.org/content/10.1101/2022.01.22.477348v2, Tweets: 65
URL: https://www.biorxiv.org/content/10.1101/2024.08.28.610193v1, Tweets: 44
URL: https://www.biorxiv.org/content/10.1101/2024.05.08.593115v2, Tweets: 37
URL: https://www.biorxiv.org/content/10.1101/2024.02.26.582181v2, Tweets: 33
URL: https://www.biorxiv.org/content/10.1101/2024.06.10.598294v4, Tweets: 28
URL: https://www.biorxiv.org/content/10.1101/2024.08.29.610422v1, Tweets: 22
URL: https://www.biorxiv.org/content/10.1101/2024.08.30.610287v1, Tweets: 20
URL: https://www.biorxiv.org/content/10.1101/2024.08.28.610199v1, Tweets: 17
URL: https://www.biorxiv.org/content/10.1101/2022.07.13.499031v2, Tweets: 16


In [1]:
from bioarxiv import get_trending_urls

top_ten_tweets = get_trending_urls()

print(top_ten_tweets)

Yesterday's date in ET: 2024-08-30
https://www.biorxiv.org/content/early/recent?page=0
page date:  2024-08-31
https://www.biorxiv.org/content/early/recent?page=1
page date:  2024-08-31
https://www.biorxiv.org/content/early/recent?page=2
page date:  2024-08-31
https://www.biorxiv.org/content/early/recent?page=3
page date:  2024-08-31
https://www.biorxiv.org/content/early/recent?page=4
page date:  2024-08-30
https://www.biorxiv.org/content/early/recent?page=5
page date:  2024-08-30
https://www.biorxiv.org/content/early/recent?page=6
page date:  2024-08-30
https://www.biorxiv.org/content/early/recent?page=7
page date:  2024-08-30
https://www.biorxiv.org/content/early/recent?page=8
page date:  2024-08-30
https://www.biorxiv.org/content/early/recent?page=9
page date:  2024-08-30
https://www.biorxiv.org/content/early/recent?page=10
page date:  2024-08-30
https://www.biorxiv.org/content/early/recent?page=11
page date:  2024-08-30
https://www.biorxiv.org/content/early/recent?page=12
page date:

In [3]:
print(top_ten_tweets)
for i in top_ten_tweets:
    print(i)

[{'url': 'https://www.biorxiv.org/content/10.1101/2023.11.27.568910v2', 'tweet_count': '103', 'abstract': 'The impact of synonymous codon choice on protein output has important implications for understanding endogenous gene expression and design of synthetic mRNAs. Synonymous codons are decoded at different speeds, but simple models predict that this should not drive protein output. Instead, translation initiation should be the rate limiting step for production of protein per mRNA, with little impact of codon choice. Previously, we used a neural network model to design a series of synonymous fluorescent reporters and showed that their protein output in yeast spanned a seven-fold range corresponding to their predicted translation elongation speed. Here, we show that this effect is not due primarily to the established impact of slow elongation on mRNA stability, but rather, that slow elongation further decreases the number of proteins made per mRNA. We combine simulations and careful exp

In [3]:
#now i need to do the ai part
import requests
import fitz  # PyMuPDF
import tiktoken
from openai import OpenAI
import os
import json
from dotenv import load_dotenv
from PIL import Image, ImageDraw, ImageFont
import textwrap
from datetime import datetime
from github import process_paper

# Load environment variables from .env file
load_dotenv()

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def download_and_extract_paper_info(paper_info, token_limit=120000, model="gpt-3.5-turbo"):
    pdf_url = paper_info['url']
    
    # Ensure the URL ends with .full.pdf
    if not pdf_url.endswith(".full.pdf"):
        pdf_url += ".full.pdf"
    
    response = requests.get(pdf_url)
    if response.status_code == 200:
        pdf_content = response.content
        
        doc = fitz.open(stream=pdf_content, filetype="pdf")
        text = ""
        encoding = tiktoken.encoding_for_model(model)
        
        for page in doc:
            page_text = page.get_text()
            text += page_text
            
            tokens = encoding.encode(text)
            if len(tokens) > token_limit:
                text = encoding.decode(tokens[:token_limit])
                break
        twitter_handles = process_paper(text)
        # this will either be an empty list or a list with twitter handles 
        return {
            "title": paper_info['title'],
            "publish_date": datetime.now().strftime('%Y-%m-%d'),  # Assuming current date as publish date
            "full_text": text,
            "twitter_handles": twitter_handles
        }
    else:
        print(f"Failed to download paper. Status code: {response.status_code}")
        return None

def summarize_text(text):
    prompt = f""" 
    You are getting the text version of an arxiv paper your goal is to provide a summary of the paper by providing bullet points which summarise the paper. 

    It should be exact three bullet points which summarise the paper. Return your response in JSON format where the keys are the bullet points and the values are the summaries of the bullet points as following:

    {{
    "bullet_point_1": "content",
    "bullet_point_2": "content",
    "bullet_point_3": "content"
    }}

    Here is the text of the paper:

    {text}
    """

    completion = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.0,
    )

    summary = completion.choices[0].message.content
    return summary

def add_text_to_image(background_path, title, text_content, publish_date, output_path="output.jpg", scale_factor=2, offset=20):
    with Image.open(background_path) as img:
        width, height = img.size
        background = img.resize((width * scale_factor, height * scale_factor), Image.LANCZOS)
    
    draw = ImageDraw.Draw(background)

    title_font = ImageFont.truetype("fonts/Inika-Regular.ttf", 35 * scale_factor)
    content_font = ImageFont.truetype("fonts/Inika-Regular.ttf", 20 * scale_factor)
    date_font = ImageFont.truetype("fonts/Inika-Regular.ttf", 20 * scale_factor)
    arxiv_font = ImageFont.truetype("fonts/Larabieb.ttf", 50 * scale_factor)

    margin = 50 * scale_factor
    max_width = background.width - (2 * margin)

    # Dynamically calculate the width for wrapping the title
    wrapped_title = textwrap.wrap(title, width=int(max_width / (35 * scale_factor * 0.6)))
    y_text = 50 * scale_factor

    for line in wrapped_title:
        bbox = title_font.getbbox(line)
        line_width = bbox[2] - bbox[0]
        line_height = bbox[3] - bbox[1]
        x_text = (background.width - line_width) // 2
        draw.text((x_text, y_text), line, font=title_font, fill=(0, 0, 0))
        y_text += line_height + (10 * scale_factor)

    bullet_points = json.loads(text_content)
    total_height = sum(len(textwrap.wrap(value, width=90)) * (25 * scale_factor) + (20 * scale_factor) for value in bullet_points.values())
    y = (background.height - total_height) // 2
    bullet_width = content_font.getbbox("• ")[2]
    max_content_width = max(max(content_font.getbbox(line)[2] for line in textwrap.wrap(value, width=90)) for value in bullet_points.values())
    bullet_start_x = (background.width - max_content_width - bullet_width) // 2

    for value in bullet_points.values():
        wrapped_text = textwrap.wrap(value, width=90)
        
        for i, line in enumerate(wrapped_text):
            if i == 0:
                draw.text((bullet_start_x, y), "•", font=content_font, fill=(0, 0, 0))
                draw.text((bullet_start_x + bullet_width, y), line, font=content_font, fill=(0, 0, 0))
            else:
                draw.text((bullet_start_x + bullet_width, y + (25 * scale_factor * i)), line, font=content_font, fill=(0, 0, 0))
        
        y += (25 * scale_factor * len(wrapped_text)) + (20 * scale_factor)

    date_text = f"Published: {publish_date}"
    date_bbox = date_font.getbbox(date_text)
    date_height = date_bbox[3] - date_bbox[1]
    draw.text((margin, background.height - margin - date_height - offset), date_text, font=date_font, fill=(0, 0, 0))

    arxiv_text = "@arXivGPT"
    arxiv_bbox = arxiv_font.getbbox(arxiv_text)
    arxiv_width = arxiv_bbox[2] - arxiv_bbox[0]
    arxiv_height = arxiv_bbox[3] - arxiv_bbox[1]
    arxiv_x = background.width - margin - arxiv_width
    arxiv_y = background.height - margin - arxiv_height - offset

    pre_x_text = "@ar"
    pre_x_width = arxiv_font.getbbox(pre_x_text)[2]
    draw.text((arxiv_x, arxiv_y), pre_x_text, font=arxiv_font, fill=(0, 0, 0))

    x_text = "X"
    x_width = arxiv_font.getbbox(x_text)[2]
    draw.text((arxiv_x + pre_x_width, arxiv_y), x_text, font=arxiv_font, fill="#B31B1B")

    post_x_text = "ivGPT"
    draw.text((arxiv_x + pre_x_width + x_width, arxiv_y), post_x_text, font=arxiv_font, fill=(0, 0, 0))

    background.save(output_path, quality=95)
    print(f"High-resolution image saved as {output_path}")

def create_image_from_paper_info(paper_info, background_path="background.jpg", output_path="output.jpg"):
    paper_details = download_and_extract_paper_info(paper_info)
    if paper_details:
        title = paper_details.get("title")
        publish_date = paper_details.get("publish_date")
        full_text = paper_details.get("full_text")
        summary = summarize_text(full_text)
        add_text_to_image(background_path, title, summary, publish_date, output_path)
        twitter_handles = paper_details.get("twitter_handles")
    return output_path, twitter_handles

# Example usage
# if __name__ == "__main__":
#     paper_info = {
#         'url': 'https://www.biorxiv.org/content/10.1101/2024.05.08.593115v2',
#         'tweet_count': '37',
#         'abstract': 'The authors have declared no competing interest.',
#         'title': 'Plasmodesmal closure elicits stress responses',
#         'subject_area': 'Plant Biology'
#     }
#     create_image_from_paper_info(paper_info)

In [1]:
from ai import create_image_from_paper_info 
# Example usage
if __name__ == "__main__":
    
    paper_info = {
        'url': 'https://www.biorxiv.org/content/10.1101/2024.05.08.593115v2',
        'tweet_count': '37',
        'abstract': 'The authors have declared no competing interest.',
        'title': 'Plasmodesmal closure elicits stress responses',
        'subject_area': 'Plant Biology'
    }
    output_path, twitter_handles = create_image_from_paper_info(paper_info)
    print(f"Image saved at: {output_path}")
    print(f"Twitter handles: {twitter_handles}")

High-resolution image saved as output.jpg
Image saved at: output.jpg
Twitter handles: []
